# Report

Summary your findings and motivate your choice of approach. A better motivation show your understanding of the lab. Dont forget to include the result from part 1!

**Name:** Erik Lidbeck\
**Date:** 2022-02-04

## Introduction
This lab introduces CNNs and the concept of data augmentation to improve performance alongside hyperparameter tuning.

## Result

### Metrics
Accuracy is often a straight forward metric to evaluate a model, but this can be misleading depending on the data. If the data is imbalanced multi-class then accuracy would not give a complete picture of what the model learns. In this case the number of squares, circles, and triangles are somewhat equal but there is a slight imbalance. If the model learns to classify the largest class then it's total accuracy might be high but it will have a low accuracy with the smaller classes. It is also important to note what accuracy we are talking about. If it's training accuracy it doesn't provide much information, and might even suggest the model is overfitted depending on validation and test.

An alternative metric for higher percision would of course be precision. Percision is better because it is relative to the class the model is predicting and not of all predictions, thereby providing an accurate performance matric even with imbalanced classes.

Accuracy for train and test was 0.9956 and 0.5378
Precision for train and test was 0.9961 and 0.5401

From these metrics we can see that the model isn't exactly good and overfitting to the training data.

### Architecture
#### Overfitting
As we know the model is already overfitting we can try to make it worse by removing the dropout layer because it acts as regularization, which is normally a good thing but it counteracts our current intent. Because the model is already overfitting we can try to let it run for longer to see how bad it can get.

By removing the dropout layer and adding an extra linear layer with 300 outputs we can see the validation precision dive down while it otherwise would remain somewhat stationary.
<center><img src='./Overfitting.png' width="600"></center>
And the validation loss is also increasing.
<center><img src='./Overfitting_loss.png' width="600"></center>

### Modify for better performance
I first tried to use a larger kernel (5x5) but quickly abandoned that because the validation loss and accuracy got worse. I then tried increasing the hidden nodes in the fully connected layers and adding an additional layer because I believed that the problem lied in the classification part of the network.

The best results I got was by simply changing the convolutional layers output channels. With a 32->128->256 output and then 256 hidden nodes I was able to get a better validation result at the cost of training accuracy. (Green is the modified model and Blue is the starting model)

<center><img src='./Modified.png' width="600"></center>
<center><img src='./Modified_loss.png' width="600"></center>

### Hyperparameter Tuning
Even if the previous model is a little better it is still overfitting and the validation loss is steadily increasing over longer epochs. Therefore I focused on adding more regularization to increase validation and test performance. The model was also highly unstable so I decreased the learning rate from 0.005 to 0.003. With regularization the validation loss didn't increase so I trained for longer to squeeze more performance out of the model. I ended up with a model with:
conv: 32->128->256
BatchNorm2d: All layers
Dropout: 0.5
Learning rate: 0.003
L2: 0.0005

(Pink is before hypertuning, Orange is with batch norm, and blue is without)
<center><img src='./Hypertune.png' width="600"></center>
<center><img src='./Hypertune_loss.png' width="600"></center>

At first I didn't get good results but after batch normalization the precision jumped up 10%. When I tested using only batch normalization it didn't yeild as good results so it must be the combination of batch normalization and regularization.

The model got ~0.714 precision on the test dataset

### Augmentation
Augmentation is an easy way to create new data from data we already have with a little modification.
I tested three different transformations: RandomHorizontalFlip, RandomColorJitter, and RandomRotation. I chose flip and rotation to see if the model was focusing on the location of figures. The color jitter was to see if colors mattered to the model.

Of the different transformations, flipping increased  test precision by ~0.07 while rotation and color jitter worsened by ~0.10 compared to tuned model. The best test result was achieved by combining flip and color jitter which increased test precision to ~0.815 but had worse validation performance than other setups.
(Gray is surprisingly the best on testing)
Transforms:
RandomHorizontalFlip(prob = 0.25)
ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3)
<center><img src='./Transform.png' width="600"></center>

Augmentation did really help the model and I think it had the greatest impact, along with regularization, because it creates new data so the model doesn't overfit. 